## 🛰️ Imágenes de Cobertura y Uso del Suelo de MapBiomas Colombia – Colección 2 (1985–2023)

La Colección 2 de MapBiomas Colombia (1985–2023) ofrece mapas anuales de cobertura y uso del suelo generados a partir de series temporales de imágenes satelitales. Estos datos están disponibles de forma pública como un **asset en Google Earth Engine (GEE)**.

### 📦 Asset disponible en GEE
- **asset:** projects/mapbiomas-public/assets/colombia/collection2/mapbiomas_colombia_collection2_integration_v1

Este asset corresponde a una **imagen multibanda**, donde cada banda representa un año específico. Por ejemplo:

- `classification_1985`
- `classification_2001`
- `classification_2022`
- `classification_2023`

---

## 🧪 Requisitos para usar en JupyterLab desde Docker

### 1. Instalar librerías necesarias dentro del contenedor

En una celda de Jupyter o desde la terminal del contenedor, es necesario ejecutar:

```bash
conda install conda-forge::earthengine-api
conda install conda-forge::geemap
conda install conda-forge::ipyleaflet
```

| 📦 Paquete           | 📝 Función                                                                 |
|----------------------|---------------------------------------------------------------------------|
| `earthengine-api`    | Para acceder al catálogo de Google Earth Engine (GEE)                     |
| `geemap`             | Para visualizar datos geoespaciales en mapas interactivos                |
| `jupyter-leaflet`    | Para mostrar mapas dentro de JupyterLab usando `geemap`                  |

### 2. 🔐 Autenticación en Google Earth Engine

Desde el notebook, ejecuta el siguiente código para autenticar tu cuenta de Google Earth Engine:

```python
import ee

# Solo es necesario la primera vez
ee.Authenticate()
ee.Initialize()
```
Esto abrirá un enlace en tu navegador donde deberás iniciar sesión con tu cuenta de Google asociada a Earth Engine.
Luego, copia el código de verificación que te aparece y pégalo en el campo solicitado del notebook.

✅ Este paso solo es necesario una vez por entorno. Si cierras el contenedor o cambias de entorno, puede que debas volver a autenticarte.


In [1]:
import sys
sys.path.append("../src")
import data_preprocessing
data_preprocessing.authenticate_earth_engine()

✅Auteticación y conexión con Google Earth Engine exitosa


In [2]:
# Aqui podrás visualizar las bandas que contiene la imágen de cobertura y uso de MapBiomas Colombia
imagen = data_preprocessing.load_mapbiomas_asset()
print (imagen.bandNames().getInfo())

['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 'classification_2017', 'classification_2018', 'classification_2019', 'classification_2020', 'classification_2021', 'classification_2022', 'classification_2023']


In [3]:
# Obtener el sistema de coordenadas de la imagen
projection = imagen.projection()
# Mostrar el sistema de coordenadas
print(projection.getInfo())

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [0.0002694945852358564, 0, -82.8156860429787, 0, -0.0002694945852358564, 16.131676377633134]}


# ✂️ Recorte y reproyección de capas vectoriales (SHP) a EPSG:
Este flujo permite recortar y reproyectar capas como departamentos, resguardos indígenas y áreas protegidas.

## 1. 📁 Cargar los archivos .shp

En esta sección es necesario cargar la ruta específica donde tiene guardado la carpeta del proyecto como parámetro de la función

In [4]:
dept, reg, resg, runap = data_preprocessing.load_geospatial_layers("/notebooks/DEFORESTACION/DATOS")

✅Finalizó la función load_geospatial_layers


## 2. 🧭 Asegurar proyecciones (planas) y recortar al área de estudio (Colombia)

In [5]:
# Reproyectar las capas
dept_9377, resg_9377, runap_9377, reg_9377 = data_preprocessing.reproject_layers_pl(dept, resg, runap, reg)

✅Finalizó la función reproject_layers


## 3 🧭 Selección del departamento de la amazonía con menor area de: Parques Nacionales Naturales y Resguardos indígenas

A partir de la exploración de los datos se busca identificar el departemento que pueda presentar mayor transformación del uso del suelo, por lo cual, se calcula la proporción de área de parques naturales y resguardos indígenas en departamentos de la región amazónica de Colombia utilizando operaciones espaciales con GeoPandas. 

- 1. Se determina qué departamentos tienen al menos un 30% de superposición con la región amazónica. 
- 2. Se calcula las áreas ocupadas por zonas protegidas (RUNAP) y resguardos (resguardos_9377) dentro de esos departamentos.
- 3. Se estima el área libre restante y se determina cuál departamento tiene mayor superficie sin protección.

In [6]:
#Exploración de los datos
dptos_area, resguardos_area, parques_area, dptos_amazonicos = data_preprocessing.dep_con_menor_area_protegida (dept_9377, resg_9377, runap_9377)

/notebooks/DEFORESTACION/notebooks/../src/data_preprocessing.py:86: UserWarning: `keep_geom_type=True` in overlay resulted in 227 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  interseccion = gpd.overlay(dept_9377, resg_9377, how='intersection')



✅ Departamento con mayor área libre dentro de la región (sin parques ni resguardos):
- Nombre: Caquetá
- Área total en región (m²): 92709074851.22
- Área protegida (m²): 0.00
- Área libre (m²): 92709074851.22
- Proporción libre: 100.00%


/notebooks/DEFORESTACION/notebooks/../src/data_preprocessing.py:102: UserWarning: `keep_geom_type=True` in overlay resulted in 6 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  resguardos_inter = gpd.overlay(resg_9377, dptos_amazonicos, how='intersection')


## 4. ✂️ Recortar capas al área de estudio

- Se tomó la decisión de seleccionar la departartamento de Caquetá, ya que, según la exploración anterior, es el que presenta menor área protegida (Parques Nacionales Naturales y Resguardos Indígenas). Sin embargo el análisis lo puede realizar para el departamento de interés cambiando el nombre tal como se especifica a continuación:

Amazonas - Antioquia - Arauca - Area en Litigio Cauca - Huila - Atlántico - Bolívar - Boyacá - Caldas - Caquetá - Casanare - Cauca - Cesar - Chocó - Cundinamarca - Córdoba - Guainía - Guaviare - Huila - La Guajira - Magdalena - Meta - Nariño - Norte de Santander - Putumayo - Quindío - Risaralda - San Andrés Providencia y Santa Catalina - Santander - Sucre - Tolima - Valle del Cauca - Vaupés - Vichada

In [7]:
depto = "Caquetá"
Departamento, resguardos_dpto, parques_dpto = data_preprocessing.recortar_capas_por_departamento(dept_9377, resg_9377, runap_9377, depto)

Departamento seleccionado: Caquetá
Cantidad de parques en el departamento: 99
Cantidad de resguardos en el departamento: 54


## ✅ 5.🧭 Reproyección de capas a 4326

Es fundamental que todas las capas involucradas en un análisis espacial compartan el mismo sistema de referencia de coordenadas (CRS, por sus siglas en inglés). Cuando las capas tienen diferentes proyecciones, pueden surgir errores en operaciones como intersecciones, uniones espaciales, análisis de distancias o generación de mapas.

En este paso, se reproyectan todas las capas vectoriales al sistema de referencia de coordenadas (CRS) del ráster, asegurando una compatibilidad total para los análisis posteriores.

In [8]:
dpto_4326, resguardos_dpto_4326, parques_dpto_4326 = data_preprocessing.reproject_layers(Departamento, resguardos_dpto, parques_dpto)

✅Finalizó la función reproject_layers


## 6. ✅ Guardar las capas recortadas y reproyectadas

En esta sección es necesario cargar la ruta específica de la carpeta donde se quiere guardar las capas recortadas

In [9]:
carpeta = r"/notebooks/DEFORESTACION/results/CAPAS_DPTO"
data_preprocessing.save_layers(dpto_4326, resguardos_dpto_4326, parques_dpto_4326, carpeta)

✅ Capas guardadas correctamente en: /notebooks/DEFORESTACION/results/CAPAS_DPTO


## 7. ✅ Recortar raster al área de estudio

In [10]:
cober_clipped = data_preprocessing.clip_raster_to_region(dpto_4326)

✅ Capas  convertidas correctamente


## 8. ✅ Exportar imágenes a google Drive (multibanda)

In [11]:
data_preprocessing.exportar_bandas_mapbiomas(cober_clipped, dpto_4326)

📅 Ingrese el año inicial (ej: 2019):  2019
📅 Ingrese el año final (ej: 2023):  2023


✅ Años seleccionados: ['2019', '2020', '2021', '2022', '2023']
✅ Bandas seleccionadas: ['classification_2019', 'classification_2020', 'classification_2021', 'classification_2022', 'classification_2023']
🚀 Exportación iniciada: Mapbiomas_from_2019_to_2023. Revisa la pestaña 'Tasks' en Earth Engine.
